In [1]:
import model as m

import torch
import torch.optim as optim
import gym
from collections import deque
import statistics
from visualize import update_viz

Setting up a new session...


In [2]:
env = gym.make("CartPole-v1")

In [3]:
policy = m.DiscretePolicy(
    n_obs = env.observation_space.shape[0],
    n_acts = env.action_space.n,
    n_hidden = 64
)

#value = m.Value(
#    n_obs = env.observation_space.shape[0],
#    n_hidden = 64
#)

In [4]:
policy_optimizer = optim.SGD(policy.parameters(), lr = 0.01)
#value_optimizer = optim.SGD(value.parameters(), lr = 0.01)

In [5]:
episode_reward = 0
last_episode_rewards = []

In [6]:
for _ in range(100000):
    trajectories = []
    state = torch.tensor([env.reset()], dtype=torch.float32)
    trajectories.append({
        "state": state,
        "action": torch.tensor([0]),
        "reward": torch.tensor([0.0]),
        "done": False
    })

    for _ in range(1000):
        if trajectories[-1]["done"]:
            env.reset()
            last_episode_rewards.append(episode_reward)
            episode_reward = 0

            if len(last_episode_rewards) == 1000:
                avg_ep_reward = statistics.mean(last_episode_rewards)
                print(avg_ep_reward)
                update_viz(avg_ep_reward, 1000)
                last_episode_rewards.clear()

        action = policy(trajectories[-1]["state"])
        state, reward, done, _ = env.step(action.item())
        #print(reward)

        trajectories.append({
            "state": torch.tensor([state], dtype=torch.float32),
            "action": action,
            "reward": torch.tensor([reward]),
            "done": done
        })

        episode_reward += reward

        #print(done, reward)

    states = torch.stack([trajectory["state"].squeeze(0) for trajectory in trajectories])
    actions = torch.stack([trajectory["action"] for trajectory in trajectories])
    done = torch.stack([torch.tensor([1.0]) if trajectory["done"] else torch.tensor([0.0]) for trajectory in trajectories])
    rewards = [trajectory["reward"] for trajectory in trajectories]

    #print(rewards)
    prev = torch.tensor([0.0])
    for i in range(len(rewards)):
        rewards[i] += prev * 0.99
        rewards[i] *= done[i - 1]
        prev = rewards[i]
    #print(rewards)
    rewards = torch.stack(rewards)
    

    #print(rewards[-1])

    #value_estimates = value(states)

    log_probs = policy.log_prob(states, actions)

    policy_loss = -(rewards * log_probs).mean()

    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()

    #value_loss = torch.pow((value_estimates - rewards), 2).mean()

    #value_optimizer.zero_grad()
    #value_loss.backward()
    #value_optimizer.step()

20.929
20.755
21.226
20.72
20.102
21.001
20.78
20.855


KeyboardInterrupt: 

In [0]:
#[trajectory["state"] for trajectory in trajectories]
